## 데이터 추출을 위한 데모 파일
- 데이터 프레임의 특정 열에서 단어를 추출하는 작업
- 자연어 전처리 프로세스를 통해 고유한 단어를 추출

In [108]:
import pandas as pd
from openpyxl import load_workbook
import os
import re
from nltk.corpus import stopwords
from collections import Counter


dir_path = os.getcwd()
dir_path = os.path.join(dir_path, "file")
print(dir_path)

c:\Users\PC\Desktop\git_repo\ML-DL-Training\Data_preprocess\workspace\file


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 1차 정리 단어 추출 작업

- 특수 문자 제거 및 숫자 제거
- 공백 기준 단어 분리
- 불용어 제거, 길이가 2이하인 단어 제거

In [13]:
# 파일 경로와 읽을 열 이름
file_name = "eclass_property.xlsx"
file_path = os.path.join(dir_path,file_name)  # 여기에 엑셀 파일 경로를 넣으세요.
column_name = 'Preferred name'  # 여기에 열 이름을 넣으세요.

# 엑셀 파일 읽기
df = pd.read_excel(file_path)

# 특수문자 및 숫자 제거 함수
def remove_special_characters(text):
    text = re.sub(r'[^A-Za-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# 공백 기준 단어 분리, 불용어 제거, 길이가 2이하인 함수
def process_text(text):
    words = text.split()
    filtered_words = [word[0].upper() + word[1:] for word in words if word.lower() not in stopwords.words('english') and len(word) > 2]
    
    return filtered_words

# 해당 열의 단어 추출 및 처리
all_words = []
for text in df[column_name].dropna():
    cleaned_text = remove_special_characters(text)
    words = process_text(cleaned_text)
    all_words.extend(words)


# 단어 빈도수 계산
word_freq = Counter(all_words)

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency'])

In [96]:
result_df.describe

<bound method NDFrame.describe of          Word  Frequency
0       FRONT        152
1        TYPE        149
2      CORPUS        303
3       Total        137
4       Front         68
...       ...        ...
7445   Foamed          1
7446     ZERO          3
7447   ADJUST          1
7448  Zeroset          1
7449     Zinc          2

[7450 rows x 2 columns]>

- 대문자 중복 단어 제거
- 축약어, 단어 분리 처리

In [176]:
column_data = result_df['Word'].dropna()

upper_data = pd.DataFrame()
lower_data = pd.DataFrame()

# 대문자 2개 이상(upper_data), 1개 이하(lower_data) 데이터 분리
upper_data['Word'] = column_data[column_data.str.contains(r'[A-Z].*[A-Z]')]             # 870개
lower_data['Word'] = column_data[~column_data.str.contains(r'[A-Z].*[A-Z]', na=False)]  # 6580개

# 소문자 통일 처리
upper_data['UP_Lower'] = upper_data['Word'].str.lower().to_frame()
lower_data['LO_Lower'] = lower_data['Word'].str.lower().to_frame()

# lower_data를 기준으로 inner merge
merged_df = pd.merge(upper_data, lower_data, left_on='UP_Lower', right_on='LO_Lower', how='inner')

# upper_data 중 중복이 아닌 데이터 분리
upper_only_data = upper_data.loc[~upper_data['UP_Lower'].isin(merged_df['UP_Lower']), ['Word']] #614개

# lower_data 중 중복이 아닌 데이터 분리
lower_only_data = lower_data.loc[~lower_data['LO_Lower'].isin(merged_df['LO_Lower']), ['Word']] #6324개

# overlap data 정리
overlap_data = merged_df[['Word_y']].rename(columns={'Word_y': 'Word'})                         #256개

# word data 정리
word_data = pd.concat([lower_only_data, overlap_data], ignore_index=True)                       #6580개
word_data['Word'] = word_data['Word'].str.lower()

In [177]:
word_data

,Word
0,total
1,phase
2,current
3,possible
4,pointer
...,...
6575,vortex
6576,width
6577,lock
6578,zero


- 표제어 변환

In [218]:
from textblob import TextBlob, Word
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet, words

import inflect

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('words')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [235]:
# Steemmer를 사용하여 단어의 원형 추출
def stemmer(word):
    ps = PorterStemmer()
    return ps.stem(word)

p = inflect.engine()

# 단수형 변환 함수 정의
def to_singular(word):
    singular_word = p.singular_noun(word)
    return singular_word if singular_word else word

EN_words = set(words.words())

# 영어 단어 인지 확인
def is_EN_word(word):
    if word in EN_words:
        return word
        

lemmatizer = WordNetLemmatizer()

# 품사 태그를 WordNet의 태그로 변환
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# 원형을 추출
def get_lemma(word):
    pos = pos_tag([word])[0][1]  # 단어의 품사 태깅
    wordnet_pos = get_wordnet_pos(pos)  # WordNet의 품사 태그로 변환
    lemma = lemmatizer.lemmatize(word, wordnet_pos)  # 원형 추출
    return lemma

In [237]:
# 데이터프레임에 단어의 원형 추출
word_result = pd.DataFrame()

# 추출 단어의 단수형 추출, nltk 내의 word에서 존재여부 파악, 중복 처리
word_result['Word'] = word_data['Word'].apply(lambda x: is_EN_word(to_singular(x))).dropna().drop_duplicates()

# 추출 단어의 어간 추출(Stemming)
word_result['Stemm_Word'] = word_result['Word'].apply(stemmer)

# 어간 추출 단어와 기존 단어의 비교
word_result['Data_Equal'] = word_result.eval('Word == Stemm_Word')

# 어간 추출 단어와 다른 단어중 nltk 내의 word에서 존재여부 파악 
Stemm_Word = word_result[word_result['Data_Equal'] == False][['Stemm_Word']]
word_result['Stem_con'] = Stemm_Word['Stemm_Word'].apply(is_EN_word)

# 최종 결과 col생성
word_result['Result_word'] = word_result.apply(
    lambda row: row['Word'] if row['Data_Equal'] else (row['Stemm_Word'] if row['Stem_con'] else row['Word']),
    axis=1
)

In [238]:
word_result

,Word,Stemm_Word,Data_Equal,Stem_con,Result_word
0,total,total,True,NaN,total
1,phase,phase,True,NaN,phase
2,current,current,True,NaN,current
3,possible,possibl,False,None,possible
4,pointer,pointer,True,NaN,pointer
...,...,...,...,...,...
6574,index,index,True,NaN,index
6575,vortex,vortex,True,NaN,vortex
6576,width,width,True,NaN,width
6578,zero,zero,True,NaN,zero


In [245]:
word_result["Result_word"].drop_duplicates().describe

<bound method NDFrame.describe of 0          total
1          phase
2        current
3       possible
4        pointer
          ...   
6565         duo
6574       index
6575      vortex
6576       width
6578        zero
Name: Result_word, Length: 3893, dtype: object>

In [240]:
# 결과 엑셀 시트에 분할 저장

result_file_path = os.path.join(dir_path,'processed_words_r.xlsx')
with pd.ExcelWriter(result_file_path) as writer:
    word_result.to_excel(writer, sheet_name='Sheet1', index=False)
    upper_only_data.to_excel(writer, sheet_name='Sheet2', index=False)

In [231]:
# 결과 엑셀 파일로 저장
result_file_path = os.path.join(dir_path,'processed_words_1.xlsx')
result_df.to_excel(result_file_path, index=False)